In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import ast
from datetime import datetime
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer,IterativeImputer

from sklearn.linear_model import LinearRegression


In [2]:
#Read in csv file
train = pd.read_csv('../Datasets/train.csv')
test = pd.read_csv('../Datasets/test.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [4]:
train.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue'],
      dtype='object')

In [5]:
train = train.drop(columns = ['belongs_to_collection','homepage', 'imdb_id',
                              'spoken_languages','original_title','overview',
                              'poster_path','status','tagline', 'title',
                              'Keywords','cast', 'crew'])
test = test.drop(columns = ['belongs_to_collection','homepage', 'imdb_id',
                              'spoken_languages','original_title','overview',
                              'poster_path','status','tagline', 'title',
                              'Keywords','cast', 'crew'])

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                2993 non-null   object 
 3   original_language     3000 non-null   object 
 4   popularity            3000 non-null   float64
 5   production_companies  2844 non-null   object 
 6   production_countries  2945 non-null   object 
 7   release_date          3000 non-null   object 
 8   runtime               2998 non-null   float64
 9   revenue               3000 non-null   int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 234.5+ KB


## Dealing with missing values

### genres, production_countries, spoken_languages

# Note for ARjun. You need to deal with missing values before you can run the following functions as is. my suggestion is that u change the functionality of the functions such that they work despite the missing values and also think of the outputs because those are what you will be dummying.

[heres lab 2.07 where you use ohe and make_column_transformer()](https://git.generalassemb.ly/kalymaan/lab-2.07-clustering/blob/main/starter-code.ipynb)

#### converting list of dictionaries which are written as strings into actual lists

In [7]:
big_dict = {'genres': 'name', 
 'production_companies': 'name', 
 'production_countries': 'name'
 #'spoken_languages': 'iso_639_1' # 'name' may be used... compare iso to original language, if same we drop
 # 'Keywords': 'name' 
 #'cast': ['gender','order'], # take the 'gender' from 'order' = 0 ******************
}



##### This is a function for the loop below it

In [8]:
def feature_extractor(l, key_we_want):
    # Input: l is the list we want to extract from.
    #        key_we_want is the key we want lol
    
    return [d[key_we_want] for d in l]

#### This is where we tutrn those crazy columns into things we can work with

In [9]:
for col_name, info_we_want in big_dict.items():
    
    
    
    train[col_name] = train[col_name][train[col_name].notna()].apply(ast.literal_eval) 
    test[col_name] = test[col_name][test[col_name].notna()].apply(ast.literal_eval) 
    
    train[col_name] = train[col_name][train[col_name].notna()].apply(lambda x: feature_extractor(x, info_we_want))
    test[col_name] =test[col_name][test[col_name].notna()].apply(lambda x: feature_extractor(x, info_we_want))
    
    
    

In [10]:
train.head()

,id,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,revenue
0,1,14000000,[Comedy],en,6.575393,"[Paramount Pictures, United Artists, Metro-Gol...",[United States of America],2/20/15,93.0,12314651
1,2,40000000,"[Comedy, Drama, Family, Romance]",en,8.248895,[Walt Disney Pictures],[United States of America],8/6/04,113.0,95149435
2,3,3300000,[Drama],en,64.299990,"[Bold Films, Blumhouse Productions, Right of W...",[United States of America],10/10/14,105.0,13092000
3,4,1200000,"[Thriller, Drama]",hi,3.174936,NaN,[India],3/9/12,122.0,16000000
4,5,0,"[Action, Thriller]",ko,1.148070,NaN,[South Korea],2/5/09,118.0,3923970


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                2993 non-null   object 
 3   original_language     3000 non-null   object 
 4   popularity            3000 non-null   float64
 5   production_companies  2844 non-null   object 
 6   production_countries  2945 non-null   object 
 7   release_date          3000 non-null   object 
 8   runtime               2998 non-null   float64
 9   revenue               3000 non-null   int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 234.5+ KB


### ok turn releasedate into unix

In [27]:
train['release_date'].iloc[2]

'10/10/14'

In [29]:
int(datetime.strptime(train['release_date'].iloc[2], "%m/%d/%y").timestamp())

1412917200

In [32]:
[int(datetime.strptime(x, "%m/%d/%y").timestamp()) for x in train['release_date']]

OSError: [Errno 22] Invalid argument

In [31]:
train['release_date'].apply(lambda x : int(datetime.strptime(x, "%m/%d/%y").timestamp()))

OSError: [Errno 22] Invalid argument

In [ ]:
train['release_date'] = train['release_date'].apply(lambda x: int(datetime.strptime(x, "%m/%d/%y").timestamp()))
test['release_date'] = test['release_date'].apply(lambda x: int(datetime.strptime(x, "%m/%d/%y").timestamp()))

## dummifying the categorical columns

Will not be dropping first in the following three categorical values because they are all multicategory(not mutually exclusive)

In [ ]:
combined_df = pd.concat([train,test],axis=0,ignore_index=True)

genres

In [ ]:
genres_dummies = pd.get_dummies(combined_df['genres'].explode(), prefix='genre')

genres_dummies = genres_dummies.groupby(genres_dummies.index).sum()

combined_df = pd.concat([combined_df,genres_dummies], axis=1)

combined_df.drop('genres', axis=1, inplace=True)

production_company

In [ ]:
production_country_dummies = pd.get_dummies(combined_df['production_countries'].explode(), prefix='prod_country')

production_country_dummies = production_country_dummies.groupby(production_country_dummies.index).sum()

combined_df = pd.concat([combined_df,production_country_dummies], axis=1)

combined_df.drop('production_countries', axis=1, inplace=True)

spoken language

In [ ]:
spoken_language_dummies = pd.get_dummies(combined_df['spoken_languages'].explode(), prefix='spoken_lang')

spoken_language_dummies = spoken_language_dummies.groupby(spoken_language_dummies.index).sum()

combined_df = pd.concat([combined_df,spoken_language_dummies], axis=1)

combined_df.drop('spoken_languages', axis=1, inplace=True)

In [ ]:
combined_df.shape

# Making a length of title column

In [ ]:
combined_df['length_of_title'] = combined_df['original_title'].str.len()


In [ ]:
combined_df.drop(columns=['original_title'], inplace=True)

Ok splitting the daatafram back into train and test

In [ ]:
object_cols = list(combined_df.select_dtypes('object').columns.values)
object_cols

In [ ]:
numeric_cols = list(combined_df._get_numeric_data().columns)

In [ ]:
len(numeric_cols)

In [ ]:
train_size = len(train)
train_dummified = combined_df[:train_size]
test_dummified = combined_df[train_size:]

In [ ]:
test_dummified = test_dummified.drop(columns=['revenue'])

In [ ]:
test_dummified.shape

In [ ]:
train_dummified.shape